# Model 4: Support Vector Classifier (SVC)
## Advanced Order Completion Prediction

### Objective:
Build a **Support Vector Classifier** to predict order completion status with improved performance over Logistic Regression.

### Why Support Vector Machine (SVM)?
- **Powerful Non-Linear Classifier**: Handles complex decision boundaries
- **Kernel Trick**: Transforms data to higher dimensions
- **Memory Efficient**: Uses subset of training points (support vectors)
- **Robust to Outliers**: Less affected by extreme values
- **Strong Theory**: Solid mathematical foundation
- **Industry Standard**: Used in fraud detection, customer analytics

### SVM vs Logistic Regression:
| Aspect | Logistic Regression | Support Vector Machine |
|--------|---------------------|------------------------|
| **Decision Boundary** | Linear only | Linear + Non-linear (kernels) |
| **Complexity** | Simple | Medium-High |
| **Training Speed** | Fast | Slower |
| **Accuracy** | Good (baseline) | Better (complex patterns) |
| **Interpretability** | High | Medium |
| **Outlier Sensitivity** | Moderate | Low |

### SVM Kernels:
- **Linear**: For linearly separable data
- **RBF (Radial Basis Function)**: Most popular, handles non-linearity
- **Polynomial**: For polynomial relationships
- **Sigmoid**: Neural network-like behavior

## Step 1: Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import time

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve
)

# Settings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")
print("🚀 Ready for Support Vector Classification!")

## Step 2: Load the Cleaned Data

In [ ]:
# Load the dataset
df = pd.read_csv('data/cleaned_final_data.csv')

print("="*60)
print("DATA LOADED SUCCESSFULLY")
print("="*60)
print(f"Dataset Shape: {df.shape}")
print(f"Total Rows: {len(df):,}")
print(f"Total Columns: {len(df.columns)}")
print("="*60)

# Check status distribution
print("\nOrder Status Distribution:")
print(df['status'].value_counts())
print("\nPercentage:")
print(df['status'].value_counts(normalize=True) * 100)

## Step 3: Create Binary Target & Prepare Features

In [ ]:
# Create binary target: 1 if complete, 0 otherwise
df['is_complete'] = (df['status'] == 'complete').astype(int)

print("Binary Target Created:")
print(df['is_complete'].value_counts())
print("\nClass Distribution:")
class_dist = df['is_complete'].value_counts(normalize=True) * 100
print(f"  Not Complete (0): {class_dist[0]:.2f}%")
print(f"  Complete (1): {class_dist[1]:.2f}%")

# Check for class imbalance
if abs(class_dist[0] - class_dist[1]) > 30:
    print("\n⚠ WARNING: Significant class imbalance detected!")
    print("   Consider using class_weight='balanced' in SVC")
else:
    print("\n✓ Classes are reasonably balanced")

### Select Features for Classification

In [ ]:
# Select features
feature_columns = [
    'price',
    'qty_ordered',
    'discount_amount',
    'month',
    'category_name_1',
    'payment_method'
]

target_column = 'is_complete'

# Create modeling dataset
df_model = df[feature_columns + [target_column]].copy()
df_model = df_model.dropna()

print(f"✓ Model dataset shape: {df_model.shape}")
print(f"✓ Features: {len(feature_columns)}")
print(f"✓ Target: {target_column}")
print(f"✓ Missing values: {df_model.isnull().sum().sum()}")

### Encode Categorical Variables

In [ ]:
# Initialize label encoders
label_encoders = {}

# Encode categorical columns
categorical_columns = ['category_name_1', 'payment_method']

for col in categorical_columns:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col].astype(str))
    label_encoders[col] = le
    print(f"✓ Encoded {col}: {df_model[col].nunique()} unique values")

print("\n✓ All categorical variables encoded!")
df_model.head()

## Step 4: Data Splitting & Feature Scaling

**⚠️ CRITICAL**: SVMs are **very sensitive** to feature scales. We must standardize features!

In [ ]:
# Separate features and target
X = df_model[feature_columns]
y = df_model[target_column]

# Split data: 80% train, 20% test (stratified to maintain class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class distribution
)

print("="*60)
print("DATA SPLIT COMPLETE")
print("="*60)
print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Testing set: {X_test.shape[0]:,} samples")
print(f"Features: {X_train.shape[1]}")

# Check class distribution in splits
print("\nClass distribution in training:")
print(y_train.value_counts(normalize=True) * 100)
print("\nClass distribution in testing:")
print(y_test.value_counts(normalize=True) * 100)

# Feature Scaling (CRITICAL FOR SVM!)
print("\n" + "="*60)
print("FEATURE SCALING (Essential for SVM)")
print("="*60)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features standardized (mean=0, std=1)")
print("\nScaling Statistics:")
print(f"  Mean before scaling: {X_train.mean().mean():.2f}")
print(f"  Mean after scaling: {X_train_scaled.mean():.6f}")
print(f"  Std before scaling: {X_train.std().mean():.2f}")
print(f"  Std after scaling: {X_train_scaled.std():.6f}")
print("="*60)

## Step 5: Train Support Vector Classifier with RBF Kernel

### Key Hyperparameters:
- **C**: Regularization parameter (higher = less regularization)
- **kernel**: Type of kernel function ('rbf', 'linear', 'poly', 'sigmoid')
- **gamma**: Kernel coefficient (higher = more influence per training point)
- **class_weight**: Handle class imbalance

In [ ]:
# Initialize Support Vector Classifier with RBF kernel
svc_model = SVC(
    kernel='rbf',              # Radial Basis Function (best for non-linear)
    C=1.0,                     # Regularization parameter
    gamma='scale',             # Kernel coefficient (auto-scaled)
    probability=True,          # Enable probability estimates
    random_state=42,           # Reproducibility
    class_weight='balanced'    # Handle class imbalance
)

print("="*60)
print("TRAINING SUPPORT VECTOR CLASSIFIER")
print("="*60)
print("\nModel Configuration:")
print(f"  Kernel: {svc_model.kernel.upper()}")
print(f"  C (Regularization): {svc_model.C}")
print(f"  Gamma: {svc_model.gamma}")
print(f"  Class Weight: {svc_model.class_weight}")
print(f"  Probability: {svc_model.probability}")
print("\nTraining in progress...")
print("(This may take 5-10 minutes for large dataset)\n")

# Start timer
start_time = time.time()

# Train the model
svc_model.fit(X_train_scaled, y_train)

# Calculate training time
training_time = time.time() - start_time

print("="*60)
print("✓ MODEL TRAINED SUCCESSFULLY!")
print("="*60)
print(f"Training time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"Support vectors: {svc_model.n_support_}")
print(f"Total support vectors: {svc_model.support_vectors_.shape[0]:,}")
print(f"Percentage of training data: {(svc_model.support_vectors_.shape[0]/len(X_train))*100:.2f}%")

## Step 6: Make Predictions

In [ ]:
# Make predictions
print("Making predictions...")
y_train_pred = svc_model.predict(X_train_scaled)
y_test_pred = svc_model.predict(X_test_scaled)

# Get probability estimates
y_train_proba = svc_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba = svc_model.predict_proba(X_test_scaled)[:, 1]

print("✓ Predictions completed!")
print(f"\nPrediction Examples (First 10):")

comparison_df = pd.DataFrame({
    'Actual': y_test[:10].values,
    'Predicted': y_test_pred[:10],
    'Probability': y_test_proba[:10].round(4),
    'Correct': (y_test[:10].values == y_test_pred[:10])
})
comparison_df['Actual'] = comparison_df['Actual'].map({0: 'Not Complete', 1: 'Complete'})
comparison_df['Predicted'] = comparison_df['Predicted'].map({0: 'Not Complete', 1: 'Complete'})
comparison_df['Correct'] = comparison_df['Correct'].map({True: '✓', False: '✗'})

print(comparison_df.to_string(index=False))

## Step 7: Model Evaluation

### Classification Metrics:
- **Accuracy**: Overall correctness
- **Precision**: True positives / (True positives + False positives)
- **Recall**: True positives / (True positives + False negatives)
- **F1-Score**: Harmonic mean of Precision and Recall
- **AUC-ROC**: Area under ROC curve (0.5-1.0)

In [ ]:
# Calculate metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_auc = roc_auc_score(y_train, y_train_proba)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_proba)

# Display results
print("="*60)
print("MODEL PERFORMANCE METRICS")
print("="*60)

print("\n📊 TRAINING SET:")
print(f"  Accuracy:  {train_accuracy:.4f}")
print(f"  Precision: {train_precision:.4f}")
print(f"  Recall:    {train_recall:.4f}")
print(f"  F1-Score:  {train_f1:.4f}")
print(f"  AUC-ROC:   {train_auc:.4f}")

print("\n📊 TESTING SET (Main Metrics):")
print(f"  Accuracy:  {test_accuracy:.4f} ⭐")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")
print(f"  AUC-ROC:   {test_auc:.4f}")

print("="*60)

# Performance assessment
print("\n💡 MODEL QUALITY ASSESSMENT:")
if test_accuracy >= 0.90:
    quality = "EXCELLENT"
elif test_accuracy >= 0.85:
    quality = "VERY GOOD"
elif test_accuracy >= 0.80:
    quality = "GOOD"
elif test_accuracy >= 0.75:
    quality = "FAIR"
else:
    quality = "NEEDS IMPROVEMENT"

print(f"  Overall Quality: {quality}")
print(f"  Correctly Classified: {test_accuracy*100:.2f}%")
print(f"  AUC Score: {test_auc:.4f}")

# Detailed classification report
print("\n\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT (Test Set)")
print("="*60)
print(classification_report(y_test, y_test_pred, 
                          target_names=['Not Complete', 'Complete'],
                          digits=4))

## Step 8: Confusion Matrix Analysis

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Visualize confusion matrix with percentages
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Absolute numbers
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, ax=ax1,
            xticklabels=['Not Complete', 'Complete'],
            yticklabels=['Not Complete', 'Complete'])
ax1.set_xlabel('Predicted', fontsize=12, fontweight='bold')
ax1.set_ylabel('Actual', fontsize=12, fontweight='bold')
ax1.set_title('Confusion Matrix (Counts)', fontsize=13, fontweight='bold')

# Percentages
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_percent, annot=True, fmt='.2f', cmap='Greens', cbar=True, ax=ax2,
            xticklabels=['Not Complete', 'Complete'],
            yticklabels=['Not Complete', 'Complete'])
ax2.set_xlabel('Predicted', fontsize=12, fontweight='bold')
ax2.set_ylabel('Actual', fontsize=12, fontweight='bold')
ax2.set_title('Confusion Matrix (Percentages)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

# Detailed breakdown
tn, fp, fn, tp = cm.ravel()
print("="*60)
print("CONFUSION MATRIX BREAKDOWN")
print("="*60)
print(f"True Negatives (TN):   {tn:,} - Correctly predicted Not Complete")
print(f"False Positives (FP):  {fp:,} - Incorrectly predicted Complete")
print(f"False Negatives (FN):  {fn:,} - Incorrectly predicted Not Complete")
print(f"True Positives (TP):   {tp:,} - Correctly predicted Complete")
print("="*60)

# Additional metrics
specificity = tn / (tn + fp)
print(f"\nSpecificity (True Negative Rate): {specificity:.4f}")
print(f"Sensitivity (Recall/True Positive Rate): {test_recall:.4f}")

## Step 9: ROC Curve Analysis

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)

# Find optimal threshold (Youden's index)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Plot ROC curve
plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'SVC (AUC = {test_auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Classifier (AUC = 0.5000)')
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='o', color='green', s=200,
           label=f'Optimal Threshold = {optimal_threshold:.3f}', zorder=5)
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('ROC Curve - Support Vector Classifier', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("="*60)
print("ROC CURVE ANALYSIS")
print("="*60)
print(f"AUC-ROC Score: {test_auc:.4f}")
print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"\nInterpretation:")
if test_auc >= 0.90:
    print("  ✓ EXCELLENT discrimination ability")
elif test_auc >= 0.80:
    print("  ✓ GOOD discrimination ability")
elif test_auc >= 0.70:
    print("  ○ FAIR discrimination ability")
else:
    print("  ✗ POOR discrimination ability")
print("="*60)

## Step 10: Model Comparison with Logistic Regression

In [ ]:
# Train Logistic Regression for comparison
from sklearn.linear_model import LogisticRegression

print("Training Logistic Regression for comparison...")
lr_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    n_jobs=-1
)
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

# Calculate LR metrics
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_precision = precision_score(y_test, lr_pred)
lr_recall = recall_score(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)
lr_auc = roc_auc_score(y_test, lr_proba)

# Comparison table
comparison = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC'],
    'Logistic Regression': [lr_accuracy, lr_precision, lr_recall, lr_f1, lr_auc],
    'Support Vector Machine': [test_accuracy, test_precision, test_recall, test_f1, test_auc],
    'Difference': [
        f"{((test_accuracy - lr_accuracy) / lr_accuracy * 100):+.2f}%",
        f"{((test_precision - lr_precision) / lr_precision * 100):+.2f}%",
        f"{((test_recall - lr_recall) / lr_recall * 100):+.2f}%",
        f"{((test_f1 - lr_f1) / lr_f1 * 100):+.2f}%",
        f"{((test_auc - lr_auc) / lr_auc * 100):+.2f}%"
    ]
})

print("\n" + "="*70)
print("MODEL COMPARISON: SVM vs LOGISTIC REGRESSION")
print("="*70)
print(comparison.to_string(index=False))
print("="*70)

# Visual comparison
fig, ax = plt.subplots(figsize=(12, 6))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
lr_scores = [lr_accuracy, lr_precision, lr_recall, lr_f1, lr_auc]
svm_scores = [test_accuracy, test_precision, test_recall, test_f1, test_auc]

x = np.arange(len(metrics))
width = 0.35

bars1 = ax.bar(x - width/2, lr_scores, width, label='Logistic Regression', 
               alpha=0.8, edgecolor='black', color='skyblue')
bars2 = ax.bar(x + width/2, svm_scores, width, label='Support Vector Machine', 
               alpha=0.8, edgecolor='black', color='lightcoral')

ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Model Performance Comparison: SVM vs Logistic Regression', 
            fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1])

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Winner announcement
print("\n🏆 MODEL COMPARISON SUMMARY:")
svm_wins = sum([test_accuracy > lr_accuracy, test_precision > lr_precision, 
               test_recall > lr_recall, test_f1 > lr_f1, test_auc > lr_auc])
print(f"   SVM wins in {svm_wins}/5 metrics")
if svm_wins >= 3:
    print("   ✓ Support Vector Machine is the WINNER!")
elif svm_wins == 2:
    print("   ≈ Models perform similarly")
else:
    print("   ✓ Logistic Regression is the WINNER!")

## Step 11: Additional Visualizations

In [ ]:
# Comprehensive visualization dashboard
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Probability Distribution
axes[0, 0].hist([y_test_proba[y_test == 0], y_test_proba[y_test == 1]], 
               bins=30, label=['Not Complete', 'Complete'], 
               alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Predicted Probability', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Distribution of Predicted Probabilities', fontsize=13, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Precision-Recall Curve
from sklearn.metrics import precision_recall_curve
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_test_proba)
axes[0, 1].plot(recall_vals, precision_vals, linewidth=2, color='purple')
axes[0, 1].set_xlabel('Recall', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Precision', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Precision-Recall Curve', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].fill_between(recall_vals, precision_vals, alpha=0.2, color='purple')

# 3. Calibration Curve (Reliability Diagram)
from sklearn.calibration import calibration_curve
fraction_of_positives, mean_predicted_value = calibration_curve(
    y_test, y_test_proba, n_bins=10
)
axes[1, 0].plot(mean_predicted_value, fraction_of_positives, 'o-', 
               linewidth=2, label='SVC', color='blue')
axes[1, 0].plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')
axes[1, 0].set_xlabel('Mean Predicted Probability', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Fraction of Positives', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Calibration Curve', fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Class-wise Performance
class_perf = pd.DataFrame({
    'Class': ['Not Complete', 'Complete'],
    'Precision': precision_score(y_test, y_test_pred, average=None),
    'Recall': recall_score(y_test, y_test_pred, average=None),
    'F1-Score': f1_score(y_test, y_test_pred, average=None)
})

class_perf.set_index('Class').plot(kind='bar', ax=axes[1, 1], 
                                   alpha=0.8, edgecolor='black', rot=0)
axes[1, 1].set_xlabel('Class', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Score', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Class-wise Performance Metrics', fontsize=13, fontweight='bold')
axes[1, 1].legend(loc='lower right', fontsize=10)
axes[1, 1].grid(True, alpha=0.3, axis='y')
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("✓ All visualizations generated successfully!")

## Step 12: Save the Model

In [ ]:
import pickle

# Save SVC model
with open('support_vector_classifier.pkl', 'wb') as file:
    pickle.dump(svc_model, file)

# Save scaler
with open('scaler_svc.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Save label encoders
with open('label_encoders_svc.pkl', 'wb') as file:
    pickle.dump(label_encoders, file)

print("="*60)
print("✓ MODEL SAVED SUCCESSFULLY!")
print("="*60)
print("Saved files:")
print("  - support_vector_classifier.pkl")
print("  - scaler_svc.pkl")
print("  - label_encoders_svc.pkl")

print("\n💡 MODEL SUMMARY:")
print(f"  Algorithm: Support Vector Classifier (RBF Kernel)")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"  AUC-ROC: {test_auc:.4f}")
print(f"  Training samples: {len(X_train):,}")
print(f"  Support vectors: {svc_model.support_vectors_.shape[0]:,}")
print(f"  Features: {len(feature_columns)}")
print("="*60)

## 🎯 Conclusion

### What We Accomplished:
1. ✅ Trained Support Vector Classifier with RBF kernel
2. ✅ Compared performance with Logistic Regression
3. ✅ Comprehensive evaluation with multiple metrics
4. ✅ ROC curve and confusion matrix analysis
5. ✅ Calibration curve and probability analysis
6. ✅ Model saved for deployment

### Key Insights:
- **Non-Linear Classification**: SVM handles complex decision boundaries
- **Kernel Trick**: Transforms data to higher dimensions
- **Memory Efficient**: Uses only support vectors
- **Robust**: Less sensitive to outliers

### SVM Advantages:
✅ **Complex Patterns**: Captures non-linear relationships  
✅ **High Dimensional**: Works well with many features  
✅ **Theoretical Foundation**: Strong mathematical basis  
✅ **Robust**: Less prone to overfitting with proper regularization  

### When to Use SVM:
- Binary classification problems
- Non-linear decision boundaries
- High-dimensional data
- Small to medium datasets
- When accuracy is critical

### SVM Limitations:
⚠ **Slow Training**: O(n²) to O(n³) complexity  
⚠ **Memory Intensive**: For very large datasets  
⚠ **Hyperparameter Sensitive**: Requires tuning  
⚠ **Black Box**: Less interpretable than logistic regression  

### Model Comparison Summary:

| Model | Pros | Cons | Best For |
|-------|------|------|----------|
| **Logistic Regression** | Fast, interpretable, simple | Linear only | Baseline, explainability |
| **Support Vector Machine** | Non-linear, robust, accurate | Slow, needs tuning | Complex patterns, accuracy |

### Next Steps:
- Hyperparameter tuning (GridSearchCV for C, gamma)
- Try different kernels (poly, sigmoid)
- Feature selection for faster training
- Ensemble with other models
- Deploy to production

---
**Congratulations on mastering Support Vector Machines! 🚀🎯**